In [1]:
from azureml.core import Workspace, Experiment


ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-123738
Azure region: southcentralus
Subscription id: 422c78cb-7b9f-41e7-a94c-e144f7afde5e
Resource group: aml-quickstarts-123738


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
import pandas as pd

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = exp.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

from azureml.core.compute_target import ComputeTargetException
cpu_cluster_name = "cpu-cluster-4"

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:

%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn
- pip:
  - azureml-defaults

Writing conda_dependencies.yml


In [8]:

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.core import ScriptRunConfig
import shutil
import os

# Specify parameter sampler
from azureml.train.hyperdrive import choice, loguniform
ps = RandomParameterSampling(
    {
        '--batch-size': choice(25, 50, 100),
        '--first-layer-neurons': choice(10, 50, 200, 300, 500),
        '--second-layer-neurons': choice(10, 50, 200, 500),
        '--learning-rate': loguniform(-6, -1)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

from azureml.core import Environment
env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

# Create a SKLearn estimator for use with train.py
shutil.copy('train.py', "./training")
est = ScriptRunConfig(source_directory= "./training",
                      script='train.py',
                      arguments=['--kernel', 'linear', '--penalty', 1.0],
                      compute_target=compute_target,
                      environment=env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_c86a0657-e495-4d57-8084-e6d9af638a40
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_c86a0657-e495-4d57-8084-e6d9af638a40?wsid=/subscriptions/422c78cb-7b9f-41e7-a94c-e144f7afde5e/resourcegroups/aml-quickstarts-123738/workspaces/quick-starts-ws-123738

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-10-30T13:36:55.982456][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-10-30T13:36:55.088061][API][INFO]Experiment created<END>\n""<START>[2020-10-30T13:36:56.298485][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-10-30T13:36:57.1592785Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())
model = best_run.register_model(model_name='train', model_path='outputs/model')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '422c78cb-7b9f-41e7-a94c-e144f7afde5e'
resource_group = 'aml-quickstarts-123738'
workspace_name = 'quick-starts-ws-123738'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='udacity-project')

ds = dataset.to_pandas_dataframe()



In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x)
scaled_data = scaler.transform(x)
scaled_data = pd.DataFrame(scaled_data, columns= x.columns)
x_train, x_test, y_train, y_test = train_test_split(scaled_data, y, test_size=0.2, random_state=1)
x_df = x_train.assign(y_train=y_train.values)
df = pd.DataFrame(x_df)
df.to_csv("training/train_data.csv", index=False)

ds = ws.get_default_datastore()
ds.upload(src_dir='./training', target_path='bankmarketing', overwrite=True, show_progress=True)

train_data = Dataset.Tabular.from_delimited_files(path=ds.path('bankmarketing/train_data.csv'))


In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_settings = {
    "experiment_timeout_minutes" : 30,
    "n_cross_validations": 2,
    "primary_metric": 'AUC_weighted',
}

automl_config = AutoMLConfig(task = 'classification',
                             training_data = train_data,
                             label_column_name = "y_train",
                             **automl_settings
                            )


In [ ]:
# Submit your automl run

remote_run = experiment.submit(automl_config, show_output = False)
remote_run.wait_for_completion()


In [ ]:
# Retrieve and save your best automl model.

best_run_customized, fitted_model_customized = remote_run.get_output()

from azureml.widgets import RunDetails
RunDetails(remote_run).show()

from azureml.core.run import Run
model_explainability_run_id = remote_run.id + "_" + "ModelExplain"
print(model_explainability_run_id)
model_explainability_run = Run(experiment=experiment, run_id=model_explainability_run_id)
model_explainability_run.wait_for_completion()

# Get the best run object
best_run, fitted_model = remote_run.get_output()
model = best_run.register_model(model_name='train', model_path='outputs/model_automl')

In [ ]:
delete()